<a href="https://colab.research.google.com/github/kaarpage/Caixeiro-Viajante/blob/main/caixeiro_viajante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema do Caixeiro Viajante
##### Nome: Júlia Almeida
##### RA: 772116

Suponha que um caixeiro viajante tenha que visitar n cidades diferentes, iniciando e
encerrando sua viagem na primeira cidade. Suponha, também, que não importa a
ordem com que as cidades são visitadas e que de cada uma delas o caixeiro pode ir
diretamente para qualquer outra. O problema do caixeiro viajante consiste em descobrir
a rota que torna mínima a viagem total.

Implemente em Python um Algoritmo Evolutivo para a resolução do problema do
Caixeiro viajante considerando 6 (seis) cidades

In [ ]:
import numpy as np
import random

### Passo 1: cidades

In [ ]:
cities = [0, 1, 2, 3, 4, 5]
cities_peso = np.zeros((6,6), dtype=int)

for i in range(6):
    for j in range(6):
        if i == j:
            cities_peso[i][j] = 0
        else:
            peso = np.random.randint(1,100)
            cities_peso[i][j] = cities_peso[j][i] = peso
cities_peso

array([[ 0,  2, 37, 10, 29, 14],
       [ 2,  0, 24, 82, 23, 67],
       [37, 24,  0, 79, 90, 43],
       [10, 82, 79,  0, 27, 97],
       [29, 23, 90, 27,  0, 97],
       [14, 67, 43, 97, 97,  0]])

### Passo 2: determinando rotas possíveis para x indivíduos

In [ ]:
ind = 10
#número de indivíduos

rotas = np.zeros((ind,7), dtype=int)
#o tamanho do array é descrito como a quantidade de indivíduos X rotas possíveis indo de 0 e retornando em 0 também

for i in range(ind):
    for j in range(6):
        city = np.random.randint(1,6)
        if j == 0 or j==6:
            rotas[i][j] = 0
        elif city in rotas[i]:    
          while city in rotas[i]:
                city = np.random.randint(1,6)
          rotas[i][j] = city
        else:
           rotas[i][j] = city
rotas

array([[0, 3, 1, 2, 5, 4, 0],
       [0, 4, 3, 2, 5, 1, 0],
       [0, 2, 5, 1, 4, 3, 0],
       [0, 3, 1, 4, 2, 5, 0],
       [0, 3, 2, 4, 1, 5, 0],
       [0, 3, 2, 1, 5, 4, 0],
       [0, 5, 4, 2, 1, 3, 0],
       [0, 1, 4, 2, 3, 5, 0],
       [0, 2, 5, 4, 1, 3, 0],
       [0, 4, 2, 1, 3, 5, 0]])

### Passo 3: criação da função fitness, que vai receber toda a população e calcular os valores fitness

In [ ]:
def fitness(ind):
    w = 0
    for i in range(len(ind)-1):
        w += cities_peso[ind[i]][ind[i+1]]
    return w
  
def retorna_fitness(populacao):
  fit = []
  for i in range(ind):
    f = fitness(populacao[i])
    fit.append(f)
  
  return fit

fit = retorna_fitness(rotas)
fit

[285, 247, 207, 262, 283, 306, 317, 305, 292, 336]

### Passo 4: Seleção por torneio

In [ ]:
def selecao_torneio(fitnesses):
  
  ind1 = -1
  ind2 = -1

  while ind1 == ind2:
    # Torneio 1
    sorteados = random.sample(range(0, ind), 2)
    if fitnesses[sorteados[0]] < fitnesses[sorteados[1]]:
      ind1 = sorteados[0]
    else:
      ind1 = sorteados[1]

    # Torneio 2
    sorteados = random.sample(range(0, ind), 2)
    if fitnesses[sorteados[0]] < fitnesses[sorteados[1]]:
      ind2 = sorteados[0]
    else:
      ind2 = sorteados[1]  
        
  return ind1,ind2    


### Passo 5: Crossover de 1 ponto

In [ ]:
def crossover(x,populacao):
  a = populacao[x[0]]
  b = populacao[x[1]]
  filho1 = a[0:2]
  for city in b:
    if not city in filho1:
      filho1 = np.concatenate((filho1,[city]))
  
  filho2 = b[0:2]
  for city in a:
    if not city in filho2:
      filho2 = np.concatenate((filho2,[city]))

  return np.concatenate((filho1,[0])),np.concatenate((filho2,[0]))

### Passo 6: Elistismo

In [ ]:
def elitismo(fitnesses):

  id1 = fitnesses.index(max(fitnesses))

  fitnesses.pop(id1)
  id2 = fitnesses.index(max(fitnesses))

  return id1,id2

### Passo 7: Mutação


In [ ]:
def mutation(children, rate):
    for child in children:
      if random.random() < rate:
        pos = [0,0]
        pos[0],pos[1] = random.randint(1, len(child)-2), random.randint(1, len(child)-2)
        while pos[0] == pos[1]:
            pos[1] = random.randint(1, len(child)-2)
        child[pos[0]], child[pos[1]] = child[pos[1]], child[pos[0]]
    return children

In [ ]:
# Número de gerações
num_geracoes = 500
for it in range(num_geracoes):

  # Nova populacao vazia
  nova_populacao = np.zeros((ind,7), dtype=int)

  # Calcula o fitness da população
  fit = retorna_fitness(rotas)

  # Imprime o melhor individuo
  id_melhor = fit.index(min(fit))

  print("Resultado:",' '.join(map(str,rotas[id_melhor])),fit[id_melhor])


  # Elitismo
  elite = elitismo(fit.copy()) # Copiando por causa da referencia do vetor
  nova_populacao[0] = rotas[elite[0]]
  nova_populacao[1] = rotas[elite[1]]

  # Gera os filhos restantes para completar a população
  num_filhos = 2
  while num_filhos < ind:

    # Seleção por torneio
    ind_vencedores = selecao_torneio(fit)
    
    # Cruzamento
    filhos = crossover(ind_vencedores,rotas)

    # Mutacao (50% de chance de mutação em cada indivíduo)
    filhos = mutation(filhos,0.5)

    # Coloca os filhos na nova população
    nova_populacao[num_filhos] = filhos[0]
    nova_populacao[num_filhos+1] = filhos[1]

    # Aumenta o número de filhos
    num_filhos = num_filhos + 2

  # Substitui a populacao antiga pela atual
  rotas = nova_populacao.copy()

Resultado: 0 2 5 1 4 3 0 207
Resultado: 0 1 4 3 2 5 0 188
Resultado: 0 3 4 2 5 1 0 239
Resultado: 0 3 4 5 2 1 0 203
Resultado: 0 3 4 5 2 1 0 203
Resultado: 0 3 2 1 4 5 0 247
Resultado: 0 5 2 1 4 3 0 141
Resultado: 0 3 4 2 1 5 0 232
Resultado: 0 3 5 2 1 4 0 226
Resultado: 0 3 4 1 2 5 0 141
Resultado: 0 1 4 3 2 5 0 188
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 5 2 1 4 3 0 141
Resultado: 0 5 2 1 4 3 0 141
Resultado: 0 2 5 1 4 3 0 207
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 5 2 3 4 1 0 188
Resultado: 0 5 2 1 4 3 0 141
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 5 2 3 4 1 0 188
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 1 2 5 4 3 0 203
Resultado: 0 4 3 2 5 1 0 247
Resultado: 0 1 4 3 2 5 0 188
Resultado: 0 4 1 5 2 3 0 251
Resultado: 0 2 1 4 3 5 0 222
Resultado: 0 5 4 1 2 3 0 247
Resultado: 0 4 1 5 2 3 0 251
Resultado: 0 1